In [1]:
import tensorflow
import pickle 
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np


2025-01-03 12:00:54.710122: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-03 12:00:54.713294: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-03 12:00:54.724900: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735884054.743581   13653 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735884054.749068   13653 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-03 12:00:54.767028: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
model = load_model('model.h5')

with open('one_hot_encoder_geo.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)

with open('Label_Encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)    

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

2025-01-03 12:00:57.468889: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [3]:
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [4]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [5]:
label_encoder_gender.classes_

array(['Female', 'Male'], dtype=object)

In [6]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [7]:
label_encoder_geo
print(label_encoder_geo.categories_)

[array(['France', 'Germany', 'Spain'], dtype=object)]


In [8]:
encoder = label_encoder_geo.transform(input_df[['Geography']]).toarray()
encoder

array([[1., 0., 0.]])

In [9]:
columns = label_encoder_geo.get_feature_names_out(['Geography'])
columns

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [10]:
df = pd.DataFrame(encoder, columns=columns)
df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [11]:
final_df = pd.concat((input_df, df), axis=1)
final_df.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [12]:
final_df = final_df.drop('Geography', axis=1)

In [13]:
final_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [14]:
input_scaled = scaler.transform(final_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [15]:
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step


array([[0.05570027]], dtype=float32)

In [16]:
prediction_prob = prediction[0][0]
prediction_prob

np.float32(0.05570027)

In [17]:
if prediction_prob > 0.5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
